## Evaluation


In [13]:
import json

# TODO: relevant = 3 or relevant = 2 || 3?
def calculate_MRR(model):
    ratings = scores["queries"]
    number_of_queries = len(ratings)

    total = 0
    for rating in ratings:
        rank_first_relevant = 0 # relevant: score = 3
        for wine in rating[model]: # loop over the 10 wines returned
            if wine["score"] == "2" or wine["score"] == "3":
                rank_first_relevant = wine["rank"]
                break

        if rank_first_relevant != 0:
            total += 1/rank_first_relevant
    
    return total/number_of_queries



with open("scores.json", "r") as file:
    scores = json.load(file)


print(f"MRR Model: {calculate_MRR('model')}")
print(f"MRR Baseline: {calculate_MRR('baseline')}")






MRR Model: 0.2777777777777778
MRR Baseline: 0.13055555555555556
